# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [2]:
%load_ext autoreload
%autoreload 2
from helpers.irp_integration import IRPClient
import json, os
from pathlib import Path

irp_client = IRPClient()

### Configurations

In [70]:
edm_name = "CBHU_Automated" # ensure unique within IRP
portfolio_name = "CBHU_Automated" # ensure unique within the edm
accounts_file_name = "Modeling_202503_Moodys_CBHU_Account.csv"
locations_file_name = "Modeling_202503_Moodys_CBHU_Location.csv"
analysis_job_name = "CBHU_Analysis"
analysis_profile_name = "DLM CBHU v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"

### Create EDM

In [71]:
print(json.dumps(irp_client.edm.create_edm(edm_name, "databridge-1"), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796554
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796554
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796554
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796554
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796554
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796554
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796554
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/risk

### Create Portfolio

In [74]:
portfolio_response = irp_client.portfolio.create_portfolio(edm_name, portfolio_name)
portfolio_id = portfolio_response['id'].split('/')[-1]
print(json.dumps(portfolio_response, indent=2))

{
  "id": "1"
}


### MRI Import

In [75]:
notebook_dir = Path(os.getcwd())
working_files_dir = notebook_dir / "workspace" / "workflows" / "_Tools" / "files" / "working_files"

bucket_response = irp_client.mri_import.create_aws_bucket()
bucket_url = bucket_response.headers['location']
bucket_id = bucket_url.split('/')[-1]

accounts_file_path = os.path.join(working_files_dir, accounts_file_name)
accounts_credentials = irp_client.mri_import.get_file_credentials(bucket_url, accounts_file_name, 100, "account")
irp_client.mri_import.upload_file_to_s3(accounts_credentials, accounts_file_path)

locations_file_path = os.path.join(working_files_dir, locations_file_name)
locations_credentials = irp_client.mri_import.get_file_credentials(bucket_url, locations_file_name, 100, "location")
irp_client.mri_import.upload_file_to_s3(locations_credentials, locations_file_path)

mapping_file_name = "mapping.json"
mapping_file_path = os.path.join(working_files_dir, mapping_file_name)
mapping_file_id = irp_client.mri_import.upload_mapping_file(mapping_file_path, bucket_id).json()

import_response = irp_client.mri_import.execute_mri_import(
    edm_name,
    int(portfolio_id),
    int(bucket_id),
    accounts_credentials['file_id'],
    locations_credentials['file_id'],
    mapping_file_id
)
print(json.dumps(import_response, indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796568
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796568
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796568
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796568
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796568
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796568
Workflow status: RUNNING; Percent complete: 100
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796568
Workflow status: FINISHED; Percent complete: 100
{
  "id": 22796568,
  "userName": "Partner - Premium

### Create Treaty

In [76]:
treaty_name = "CBHU_Treaty"
treaty_type_name = "Working Excess"
currency_name = "US Dollar"
attachment_basis = "Losses Occurring"
attachment_level = "Location"

cedant_data = irp_client.edm.get_cedants_by_edm(edm_name)["searchItems"]
lob_data = irp_client.edm.get_lobs_by_edm(edm_name)["searchItems"]

treaty_type_data = irp_client.treaty.get_treaty_types_by_edm(edm_name)["entityItems"]["values"]
treaty_type_match = next(
    (item for item in treaty_type_data if item["name"] == treaty_type_name),
    None  # fallback if no match found
)

attachment_basis_data = irp_client.treaty.get_treaty_attachment_bases_by_edm(edm_name)["entityItems"]["values"]
attachment_basis_match = next(
    (item for item in attachment_basis_data if item["name"] == attachment_basis),
    None  # fallback if no match found
)

attachment_level_data = irp_client.treaty.get_treaty_attachment_levels_by_edm(edm_name)["entityItems"]["values"]
attachment_level_match = next(
    (item for item in attachment_level_data if item["name"] == attachment_level),
    None  # fallback if no match found
)

currency_data = irp_client.reference_data.get_currencies()["entityItems"]["values"]
currency_match = next(
    (item for item in currency_data if item["name"] == currency_name),
    None  # fallback if no match found
)

if len(cedant_data) > 0 and len(lob_data) > 0 and treaty_type_match and attachment_basis_match and attachment_level_match and currency_match:
    data = {
        "treatyNumber": treaty_name,
        "treatyName": treaty_name,
        "treatyType": treaty_type_match,
        "riskLimit": 3000000,
        "occurLimit": 9000000,
        "attachPt": 2000000,
        "cedant": cedant_data[0],
        "effectDate": "2025-10-15T17:49:10.637Z",
        "expireDate": "2026-10-15T17:49:10.637Z",
        "currency": {'code': currency_match['code'], 'name': currency_match['name']},
        "attachBasis": attachment_basis_match,
        "attachLevel": attachment_level_match,
        "pcntCovered": 100,
        "pcntPlaced": 95,
        "pcntRiShare": 100,
        "pcntRetent": 100,
        "premium": 0,
        "numOfReinst": 99,
        "reinstCharge": 0,
        "aggregateLimit": 0,
        "aggregateDeductible": 0,
        "priority": 1,
        "retentAmt": "",
        "isValid": True,
        "userId1": "",
        "userId2": "",
        "maolAmount": "",
        "lobs": lob_data,
        "tagIds": []
    }

    treaty_response = irp_client.treaty.create_treaty(edm_name, data)
    lob_ids = []
    for lob in lob_data:
        lob_ids.append(lob['id'])
    print(json.dumps(irp_client.treaty.assign_lobs(edm_name, treaty_response['id'], lob_ids), indent=2))
    

[
  {
    "context": {
      "headers": {},
      "entityAnnotations": [],
      "entityStream": {
        "closed": false,
        "committed": false
      },
      "length": -1,
      "allowedMethods": [],
      "links": [],
      "stringHeaders": {},
      "acceptableMediaTypes": [
        {
          "type": "*",
          "subtype": "*",
          "parameters": {},
          "quality": 1000,
          "wildcardType": true,
          "wildcardSubtype": true
        }
      ],
      "acceptableLanguages": [
        "*"
      ],
      "committed": false,
      "requestCookies": {},
      "lengthLong": -1,
      "responseCookies": {}
    },
    "status": 201,
    "length": -1,
    "statusInfo": "CREATED",
    "metadata": {},
    "allowedMethods": [],
    "cookies": {},
    "links": [],
    "stringHeaders": {},
    "headers": {}
  }
]


### Upgrade EDM Version

In [77]:
print(json.dumps(irp_client.edm.upgrade_edm_version(edm_name), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796593
Workflow status: PENDING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796593
Workflow status: RUNNING; Percent complete: 100
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796593
Workflow status: FINISHED; Percent complete: 100
{
  "id": 22796593,
  "userName": "Partner - PremiumIQ key",
  "status": "FINISHED",
  "submitTime": "2025-10-17T02:25:38.284Z",
  "startTime": "2025-10-17T02:25:39Z",
  "endTime": "2025-10-17T02:25:49Z",
  "name": "CBHU_Automated",
  "type": "EDM_DATA_UPGRADE",
  "progress": 100,
  "priority": "medium",
  "entitlement": "RI-RISKMODELER",
  "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
  "jobs": [
    {
      "id": "7472f373-4009-4fe6-84b2-7fb5b17ad866",
      "taskId": 1,
      "workflowId": 22796593,
      "status": "Succeeded",
      "submitTime": "202

### GeoHaz

In [78]:
print(json.dumps(
    irp_client.portfolio.geohaz_portfolio(edm_name, 
                                          portfolio_id, 
                                          geocode=True, 
                                          hazard_eq=True, 
                                          hazard_ws=True), 
    indent=2
))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796601
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796601
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796601
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796601
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796601
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796601
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796601
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/risk

### Execute Single Analysis

In [65]:
analysis_response = irp_client.analysis.execute_analysis(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name,
    [treaty_response['id']]
)
print(json.dumps(analysis_response, indent=2))

{
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "edm": "CBHU",
  "eventRateSchemeId": 739,
  "exposureType": "PORTFOLIO",
  "id": "3",
  "modelProfileId": 4418,
  "outputProfileId": 123,
  "treaties": [
    "1"
  ],
  "globalAnalysisSettings": {
    "franchiseDeductible": false,
    "minLossThreshold": "1.00",
    "treatConstructionOccupancyAsUnknown": true,
    "numMaxLossEvent": 1
  },
  "jobName": "CBHU_Analysis"
}
Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793709
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793709
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793709
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22793709
Wor

### Execute Multiple Analyses

In [90]:
analysis_profile_name = "DLM CBHU v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
tag = "cbhu_analyses"

analysis_job_name1 = "CBHU_Analysis 2023 v2"
event_rate_scheme_name1 = "RMS 2023 Stochastic Event Rates"
workflow_id1 = irp_client.analysis.submit_analysis_job(
    analysis_job_name1,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name1,
    [treaty_response['id']],
    [tag]
)

analysis_job_name2 = "CBHU_Analysis 2025 v2"
event_rate_scheme_name2 = "RMS 2025 Stochastic Event Rates"
workflow_id2 = irp_client.analysis.submit_analysis_job(
    analysis_job_name2,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name2,
    [treaty_response['id']],
    [tag]
)

workflow_ids = [workflow_id1, workflow_id2]
response = irp_client.analysis.poll_analysis_job_batch(workflow_ids)
print(json.dumps(response, indent=2))
analysis_ids = []
for workflow in response['workflows']:
    analysis_ids.append(workflow['output']['analysisId'])

{
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "edm": "CBHU_Automated",
  "eventRateSchemeId": 578,
  "exposureType": "PORTFOLIO",
  "id": "1",
  "modelProfileId": 4418,
  "outputProfileId": 123,
  "treaties": [
    "1"
  ],
  "tagIds": [
    1205
  ],
  "globalAnalysisSettings": {
    "franchiseDeductible": false,
    "minLossThreshold": "1.00",
    "treatConstructionOccupancyAsUnknown": true,
    "numMaxLossEvent": 1
  },
  "jobName": "CBHU_Analysis 2023 v2"
}
{
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "edm": "CBHU_Automated",
  "eventRateSchemeId": 739,
  "exposureType": "PORTFOLIO",
  "id": "1",
  "modelProfileId": 4418,
  "outputProfileId": 123,
  "treaties": [
    "1"
  ],
  "tagIds": [
    1205
  ],
  "globalAnalysisSettings": {
    "franchiseDeductible": false,
    "minLossThreshold": "1.00",
    "treatConstructionOccupancyAsUnknown

### Group Analyses

In [91]:
group_name = "CBHU Analysis Group"
print(json.dumps(irp_client.analysis.create_analysis_group(analysis_ids, group_name), indent=2))

{
  "analysisIds": [
    "33199",
    "33200"
  ],
  "name": "CBHU Analysis Group",
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "simulateToPLT": true,
  "numOfSimulations": 50000,
  "propagateDetailedLosses": false,
  "reportingWindowStart": "01/01/2021",
  "simulationWindowStart": "01/01/2021",
  "simulationWindowEnd": "12/31/2021",
  "regionPerilSimulationSet": [],
  "description": ""
}
Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796775
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796775
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796775
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22796775
Workflow status: RUNNING; Per

### Export Analyses to RDM

In [11]:
rdm_name = "CBHU_Automated"
export_response = irp_client.rdm.export_analyses_to_rdm(rdm_name, [33198, 33197])
print(json.dumps(export_response, indent=2))

{
  "exportType": "RDM_DATABRIDGE",
  "resourceType": "analyses",
  "resourceUris": [
    "/platform/riskdata/v1/analyses/3095325",
    "/platform/riskdata/v1/analyses/3095304"
  ],
  "settings": {
    "serverId": 88094,
    "rdmName": "CBHU_Automated"
  }
}
Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/platform/export/v1/jobs/22797452
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/platform/export/v1/jobs/22797452
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/platform/export/v1/jobs/22797452
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/platform/export/v1/jobs/22797452
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/platform/export/v1/jobs/22797452
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/platform/exp